In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import re
import math
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor, XGBClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
import lightgbm as lgb
import xgboost as xgb
import catboost as cab

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, StackingRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, StackingClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm

import matplotlib.pyplot as plt
from collections import defaultdict, Counter
import warnings
import json 
import pickle
warnings.filterwarnings('ignore')

In [2]:
import pickle 
import json
from tqdm import tqdm

In [3]:
test_preds = pickle.load(open('data/test_preds_all_new.pkl', 'rb'))
train_preds = pickle.load(open('data/train_preds_all_new.pkl', 'rb'))

In [12]:
product2id = json.load(open('data/product2id.json', 'r'))
id2product = json.load(open('data/id2product.json', 'r'))
id2product = {int(k): v for k, v in id2product.items()}

In [13]:
len(train_preds), len(test_preds)

(3606249, 316971)

In [14]:
train_preds_encoded = []
for x in tqdm(train_preds, total=len(train_preds)):
    train_preds_encoded.append([product2id[id_] for id_ in x])

100%|██████████| 3606249/3606249 [02:55<00:00, 20585.36it/s]


In [15]:
test_preds_encoded = []
for x in tqdm(test_preds, total=len(test_preds)):
    test_preds_encoded.append([product2id[id_] for id_ in x])

100%|██████████| 316971/316971 [00:12<00:00, 24711.90it/s]


In [16]:
len(train_preds_encoded), len(test_preds_encoded)

(3606249, 316971)

In [17]:
pickle.dump(train_preds_encoded, open('./data/train_preds_all_encoded_new.pkl', 'wb'))
pickle.dump(test_preds_encoded, open('./data/test_preds_all_encoded_new.pkl', 'wb'))

数据读取

In [4]:
# (3606249, 3)
df_train = pd.read_csv('data/sessions_train.csv')
df_train

,prev_items,next_item,locale
0,['B09W9FND7K' 'B09JSPLN1M'],B09M7GY217,DE
1,['B076THCGSG' 'B007MO8IME' 'B08MF65MLV' 'B001B...,B001B4THSA,DE
2,['B0B1LGXWDS' 'B00AZYORS2' 'B0B1LGXWDS' 'B00AZ...,B0767DTG2Q,DE
3,['B09XMTWDVT' 'B0B4MZZ8MB' 'B0B7HZ2GWX' 'B09XM...,B0B4R9NN4B,DE
4,['B09Y5CSL3T' 'B09Y5DPTXN' 'B09FKD61R8'],B0BGVBKWGZ,DE
...,...,...,...
3606244,['B086CYFSKW' 'B0874F9859' 'B086CYFSKW'],B07B5TYD76,IT
3606245,['B09NRZKZ7V' 'B08WJTPV93'],B08L1P4C3D,IT
3606246,['B085JFX7MP' 'B085JGHW8R'],B01MPWVD44,IT
3606247,['B00B0UING2' 'B00B0UING2'],B00D3HYEZ4,IT


In [5]:
df_test_phase1 = pd.read_csv('data/sessions_test_task2.csv')
df_test_phase1

,prev_items,locale
0,['B08GYKNCCP' 'B08HCPTMJG' 'B08HCHS64Y'],ES
1,['B08NYF9MBQ' 'B085NGXGWM'],ES
2,['B091FL1QFK' 'B0B1DG29F4'],ES
3,['B004APAHCW' 'B07JMF49HN' 'B004APAHCW' 'B07JM...,ES
4,['B09YM11D4T' 'B0B12QWP5G' 'B08YMT6Q4X' 'B09V1...,ES
...,...,...
34683,['B095W5YY7B' 'B09SGL4XKB'],IT
34684,['B0BDJZWPLQ' 'B0BDJZWPLQ'],IT
34685,['B07HKMB622' 'B00FBESXOK' 'B07HKMB622' 'B00FB...,IT
34686,['B09V4J39QY' 'B0BJMSH4VX' 'B0BJMRVQFT' 'B0BJM...,IT


In [6]:
df_test_phase2 = pd.read_csv('data/phase2/sessions_test_task2.csv')
df_test_phase2

,prev_items,locale
0,['B07GTS7SWK' 'B07GTS7SWK'],ES
1,['B0B33YWVHR' '849988993X' 'B09QQG85HM' 'B0BJZ...,ES
2,['B08FMPXDTJ' 'B0B4612MTM' 'B0B45YR21M' 'B0B4F...,ES
3,['B07R3W4XQ7' 'B07R3FB5B5'],ES
4,['B09V4KBWPL' 'B09G9FTLPB'],ES
...,...,...
34685,['B089F3YK6X' 'B01BBW9TEI' 'B01BBW9Q46'],IT
34686,['B078XYC42V' 'B09TW31SND'],IT
34687,['B07ZG3V3X2' 'B09PGKF9Y8'],IT
34688,['B0B3JRGZLM' 'B09SHQJ8NV'],IT


In [7]:
def str2list(x):
    x = x.replace('[', '').replace(']', '').replace("'", '').replace('\n', ' ').replace('\r', ' ')
    l = [i for i in x.split() if i]
    return l

In [8]:
# 不到1min (1551057, 11)
products = pd.read_csv('data/products_train.csv')
products

,id,locale,title,price,brand,color,size,model,material,author,desc
0,B005ZSSN10,DE,RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wo...,30.95,RED DRAGON,NaN,NaN,RDD0089,NaN,NaN,Amberjacks Steel Dartpfeile sind verfügbar in ...
1,B08PRYN6LD,DE,Simply Keto Lower Carb* Schokodrops ohne Zucke...,17.90,Simply Keto,NaN,750 g (1er Pack),NaN,NaN,NaN,🌱 NATÜRLICHE SÜSSE DURCH ERYTHRIT - Wir stelle...
2,B09MBZJ48V,DE,"Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...",68.89,Sennheiser,Multi-Colour,One size,508377,Kunstleder,NaN,3.5 MM BUCHSE - Kann problemlos an Geräte mit ...
3,B08ZN6F26S,DE,AmyBenton Auto ab 1 2 3 ahre - Baby Aufziehbar...,18.99,Amy & Benton,Animal Car,NaN,2008B,aufziehauto 1 jahr,NaN,【Auto aufziehbar】: Drücken Sie einfach leicht ...
4,B094DGRV7D,DE,PLAYMOBIL - 70522 - Cavaliere mit grauem Pony,7.17,PLAYMOBIL,Nicht Zutreffend.,OneSize,70522,Polypropylen,NaN,Inhalt: 1 Stück
...,...,...,...,...,...,...,...,...,...,...,...
1551052,B09BW5CDRR,IT,Barbie - Playset Gelateria con Bambola con Mac...,20.48,Barbie,NaN,NaN,HCN46,NaN,NaN,DETTAGLI REALISTICI. Basta inserire la pasta m...
1551053,B0050IILBM,IT,"Braun Silk-épil 1 Depilatore Donna, Epilatore ...",22.61,Braun,Pink,NaN,4210201656067,NaN,NaN,Alimentato a corrente per un comodo utilizzo
1551054,B07W4C5W9D,IT,BoxLegend Sacchetti Sottovuoto Vestiti 6 Pezzi...,14.99,BoxLegend,6 Pezzi.,6 Pezzi (2L + 2M + 2S),6186666487608_SML,Polietilene Ppa,NaN,6 Sacchetti in 3 Diverse Misure- Questo set di...
1551055,B012D0HJXA,IT,Trasportino Pratiko Metal - Accessorio da viag...,18.35,MPS,verde,NaN,NaN,Metallo,NaN,TRASPORTINO 48X31.5X33CM


In [9]:
# 1min多一些
df_train['prev_items'] = df_train['prev_items'].apply(lambda x: str2list(x))
df_test_phase1['prev_items'] = df_test_phase1['prev_items'].apply(lambda x: str2list(x))
df_test_phase2['prev_items'] = df_test_phase2['prev_items'].apply(lambda x: str2list(x))

In [10]:
df_train

,prev_items,next_item,locale
0,"[B09W9FND7K, B09JSPLN1M]",B09M7GY217,DE
1,"[B076THCGSG, B007MO8IME, B08MF65MLV, B001B4TKA0]",B001B4THSA,DE
2,"[B0B1LGXWDS, B00AZYORS2, B0B1LGXWDS, B00AZYORS...",B0767DTG2Q,DE
3,"[B09XMTWDVT, B0B4MZZ8MB, B0B7HZ2GWX, B09XMTWDV...",B0B4R9NN4B,DE
4,"[B09Y5CSL3T, B09Y5DPTXN, B09FKD61R8]",B0BGVBKWGZ,DE
...,...,...,...
3606244,"[B086CYFSKW, B0874F9859, B086CYFSKW]",B07B5TYD76,IT
3606245,"[B09NRZKZ7V, B08WJTPV93]",B08L1P4C3D,IT
3606246,"[B085JFX7MP, B085JGHW8R]",B01MPWVD44,IT
3606247,"[B00B0UING2, B00B0UING2]",B00D3HYEZ4,IT


In [11]:
df_train['last_item'] = df_train['prev_items'].apply(lambda x: x[-1])
df_test_phase1['last_item'] = df_test_phase1['prev_items'].apply(lambda x: x[-1])
df_test_phase2['last_item'] = df_test_phase2['prev_items'].apply(lambda x: x[-1])

In [12]:
df_train.shape, df_test_phase1.shape, df_test_phase2.shape 

((3606249, 4), (34688, 3), (34690, 3))

In [13]:
loc2id = {
    'ES': 0,
    'FR': 1,
    'IT': 2,
}

In [11]:
# df_train[df_train['locale'].isin(['ES', 'FR', 'IT'])].shape

(333533, 4)

In [23]:
# df_train = df_train[df_train['locale'].isin(['ES', 'FR', 'IT'])]
# df_train.shape 

(333533, 4)

In [24]:
# df_test = df_test[df_test['locale'].isin(['ES', 'FR', 'IT'])]
# df_test.shape 

(34688, 4)

In [25]:
# df_train.shape , df_test.shape 

((333533, 4), (34688, 4))

In [13]:
# 后边不用重跑

In [14]:
# train_ids = []
# for _, row in tqdm(df_train.iterrows(), total=len(df_train)):
#     train_ids += row['prev_items']
#     train_ids.append(row['next_item'])
# train_ids = set(train_ids)
# len(train_ids)

100%|██████████| 3272716/3272716 [03:20<00:00, 16339.42it/s]


1326772

In [15]:
# test_ids = []
# for _, row in tqdm(df_test.iterrows(), total=len(df_test)):
#     test_ids += row['prev_items']
# test_ids = set(test_ids)
# len(test_ids)

100%|██████████| 316971/316971 [00:17<00:00, 17624.61it/s]


224336

In [16]:
# len(train_ids & test_ids)

212261

In [17]:
# len(train_ids | test_ids)

1338847

In [18]:
# train_ids = np.array(list(train_ids))
# test_ids = np.array(list(test_ids))
# np.save('data/train_ids.npy', train_ids)
# np.save('data/test_ids.npy', test_ids)

# 召回

召回策略

1. 使用vid关联规则【统计】
    - 计算与最后vid关联性最大的下一个vid；
    - 计算与最后5个vid关联性最大的下一个vid；
    - 计算与历史观看时间最长的vid关联性最大的下一个vid；
2. 使用vid计算词向量相似度

训练集标签中最热门的200个id

In [13]:
# 最热门的200个id
hot200 = df_train['next_item'].value_counts().head(200).index
hot200 = list(hot200)

一阶共现关系

In [107]:
window_one_item_dict = defaultdict(list)

for _, row in tqdm(df_train.iterrows(), total=len(df_train)):
    prev_items = row['prev_items']
    next_item = row['next_item']
    prev_items_length = len(prev_items)
    if prev_items_length <= 1:
        window_one_item_dict[prev_items[0]].append(next_item)
        window_one_item_dict[next_item].append(prev_items[0])
    else:
        for i, item in enumerate(prev_items[:-1]):
            window_one_item_dict[item].append(prev_items[i+1])
            window_one_item_dict[prev_items[i+1]].append(item)
        window_one_item_dict[prev_items[-1]].append(next_item)
        window_one_item_dict[next_item].append(prev_items[-1])

100%|██████████| 3606249/3606249 [04:42<00:00, 12768.91it/s]


In [68]:
# 去除泄露
window_one_item_dict = defaultdict(list)

for _, row in tqdm(df_train.iterrows(), total=len(df_train)):
    prev_items = row['prev_items']
    prev_items_length = len(prev_items)
    if prev_items_length <= 1:
        continue
    else:
        for i, item in enumerate(prev_items[:-1]):
            window_one_item_dict[item].append(prev_items[i+1])
            window_one_item_dict[prev_items[i+1]].append(item)

100%|██████████| 3606249/3606249 [04:20<00:00, 13839.73it/s]


In [108]:
for _, row in tqdm(df_test_phase1.iterrows(), total=len(df_test_phase1)):
    prev_items = row['prev_items']
    prev_items_length = len(prev_items)
    if prev_items_length <= 1:
        continue
    else:
        for i, item in enumerate(prev_items[:-1]):
            window_one_item_dict[item].append(prev_items[i+1])
            window_one_item_dict[prev_items[i+1]].append(item)

100%|██████████| 34688/34688 [00:02<00:00, 14206.04it/s]


In [109]:
for _, row in tqdm(df_test_phase2.iterrows(), total=len(df_test_phase2)):
    prev_items = row['prev_items']
    prev_items_length = len(prev_items)
    if prev_items_length <= 1:
        continue
    else:
        for i, item in enumerate(prev_items[:-1]):
            window_one_item_dict[item].append(prev_items[i+1])
            window_one_item_dict[prev_items[i+1]].append(item)

100%|██████████| 34690/34690 [00:02<00:00, 14269.24it/s]


In [110]:
pickle.dump(window_one_item_dict, open('data/window_one_item_dict_onlytask2_phase2.pkl', 'wb'))

In [111]:
window_one_item_map = {}

for item in tqdm(window_one_item_dict):
    counter = Counter(window_one_item_dict[item])
    window_one_item_map[item] = [i[0] for i in counter.most_common(100)]

100%|██████████| 1396388/1396388 [00:16<00:00, 83609.13it/s] 


In [112]:
pickle.dump(window_one_item_map, open('data/window_one_item_map_onlytask2_phase2.pkl', 'wb'))

In [113]:
k = []
v = []

for item in window_one_item_map:
    k.append(item)
    v.append(window_one_item_map[item])
    
df_next = pd.DataFrame({'item': k, 'next_item': v})
df_next = df_next.explode('next_item').reset_index(drop=True)
df_next

,item,next_item
0,B09W9FND7K,B09JSPLN1M
1,B09W9FND7K,B09W9FND7K
2,B09W9FND7K,B0828Z4D1R
3,B09W9FND7K,B00AI7MFQ2
4,B09W9FND7K,B09P83WGQG
...,...,...
14332548,B0B77ZJKD2,B08MFH8314
14332549,B0B77ZJKD2,B0B781WMXW
14332550,B00VZZ7VVA,B00V3MQ1PC
14332551,B093KK7YLB,B08K81325J


In [114]:
top200_window_one = df_next['next_item'].value_counts().index.tolist()[:200]  # 所有下一跳候选集中最热门的200个

In [115]:
pickle.dump(top200_window_one, open('data/top200_window_one_onlytask2_phase2.pkl', 'wb'))

一阶下一跳

In [124]:
next_item_dict = defaultdict(list)

for _, row in tqdm(df_train.iterrows(), total=len(df_train)):
    prev_items = row['prev_items']
    next_item = row['next_item']
    prev_items_length = len(prev_items)
    if prev_items_length <= 1:
        next_item_dict[prev_items[0]].append(next_item)
    else:
        for i, item in enumerate(prev_items[:-1]):
            next_item_dict[item].append(prev_items[i+1])
        next_item_dict[prev_items[-1]].append(next_item)

100%|██████████| 3606249/3606249 [04:51<00:00, 12365.11it/s]


In [89]:
# 去除泄露
next_item_dict = defaultdict(list)

for _, row in tqdm(df_train.iterrows(), total=len(df_train)):
    prev_items = row['prev_items']
    prev_items_length = len(prev_items)
    if prev_items_length <= 1:
        continue
    else:
        for i, item in enumerate(prev_items[:-1]):
            next_item_dict[item].append(prev_items[i+1])

100%|██████████| 3606249/3606249 [04:10<00:00, 14388.61it/s]


In [125]:
for _, row in tqdm(df_test_phase1.iterrows(), total=len(df_test_phase1)):
    prev_items = row['prev_items']
    prev_items_length = len(prev_items)
    if prev_items_length <= 1:
        continue
    else:
        for i, item in enumerate(prev_items[:-1]):
            next_item_dict[item].append(prev_items[i+1])

100%|██████████| 34688/34688 [00:02<00:00, 13937.90it/s]


In [126]:
for _, row in tqdm(df_test_phase2.iterrows(), total=len(df_test_phase2)):
    prev_items = row['prev_items']
    prev_items_length = len(prev_items)
    if prev_items_length <= 1:
        continue
    else:
        for i, item in enumerate(prev_items[:-1]):
            next_item_dict[item].append(prev_items[i+1])

100%|██████████| 34690/34690 [00:02<00:00, 14098.57it/s]


In [127]:
pickle.dump(next_item_dict, open('data/next_item_dict_onlytask2_phase2.pkl', 'wb'))

In [13]:
# next_item_dict = pickle.load(open('data/next_item_dict_new.pkl', 'rb'))

In [128]:
next_item_map = {}

for item in tqdm(next_item_dict):
    counter = Counter(next_item_dict[item])
    next_item_map[item] = [i[0] for i in counter.most_common(100)]

100%|██████████| 1326075/1326075 [00:13<00:00, 101598.88it/s]


In [129]:
pickle.dump(next_item_map, open('data/next_item_map_onlytask2_phase2.pkl', 'wb'))

In [15]:
# next_item_map = pickle.load(open('data/next_item_map.pkl', 'rb'))

一阶下一跳中最热门的200个id

In [130]:
k = []
v = []

for item in next_item_dict:
    k.append(item)
    v.append(next_item_dict[item])
    
df_next = pd.DataFrame({'item': k, 'next_item': v})
df_next = df_next.explode('next_item').reset_index(drop=True)
df_next

,item,next_item
0,B09W9FND7K,B09JSPLN1M
1,B09W9FND7K,B09JSPLN1M
2,B09W9FND7K,B09JSPLN1M
3,B09W9FND7K,B09JSPLN1M
4,B09W9FND7K,B078WW2WN5
...,...,...
15502041,B0B77ZJKD2,B0B781WMXW
15502042,B00VZZ7VVA,B00V3MQ1PC
15502043,B00VZZ7VVA,B00V3MQ1PC
15502044,B07KKKNB32,B07KKKNB32


In [131]:
top200 = df_next['next_item'].value_counts().index.tolist()[:200]  # 所有下一跳候选集中最热门的200个

In [132]:
pickle.dump(top200, open('data/top200_onlytask2_phase2.pkl', 'wb'))

In [16]:
# top200 = pickle.load(open('data/top200.pkl', 'rb'))

训练集本地验证结果

In [15]:
next_item_dict = pickle.load(open('data/window_one_item_dict_onlytask2_phase2_noleak.pkl', 'rb'))
next_item_map = pickle.load(open('data/window_one_item_map_onlytask2_phase2_noleak.pkl', 'rb'))
top200 = pickle.load(open('data/top200_window_one_onlytask2_phase2_noleak.pkl', 'rb'))

In [16]:
# df_train['last_item'] = df_train['prev_items'].apply(lambda x: x[-1])
df_train['next_item_prediction'] = df_train['last_item'].map(next_item_map)
df_train

,prev_items,next_item,locale,last_item,next_item_prediction
0,"[B09W9FND7K, B09JSPLN1M]",B09M7GY217,DE,B09JSPLN1M,"[B09W9FND7K, B09JSPLN1M, B098MKBJ2F, B09YS6S48..."
1,"[B076THCGSG, B007MO8IME, B08MF65MLV, B001B4TKA0]",B001B4THSA,DE,B001B4TKA0,"[B001B4THSA, B08MF65MLV, B0001NPYZ2, B01N3LKN0..."
2,"[B0B1LGXWDS, B00AZYORS2, B0B1LGXWDS, B00AZYORS...",B0767DTG2Q,DE,B00AZYORS2,"[B0B1LGXWDS, B00AZYORS2, B01MU5VASL, B0002TTRO..."
3,"[B09XMTWDVT, B0B4MZZ8MB, B0B7HZ2GWX, B09XMTWDV...",B0B4R9NN4B,DE,B0B71CHT1L,"[B09XMTWDVT, B0B7HZ2GWX, B0B71GSJ2R, B0B4MZZ8M..."
4,"[B09Y5CSL3T, B09Y5DPTXN, B09FKD61R8]",B0BGVBKWGZ,DE,B09FKD61R8,"[B09FKF5W9Q, B09Y5DPTXN, B09MRVWQ3B, B09FKDPY2S]"
...,...,...,...,...,...
3606244,"[B086CYFSKW, B0874F9859, B086CYFSKW]",B07B5TYD76,IT,B086CYFSKW,"[B0874F9859, B08FJQFPLM]"
3606245,"[B09NRZKZ7V, B08WJTPV93]",B08L1P4C3D,IT,B08WJTPV93,"[B08WJTPV93, B09NRZKZ7V, B088N7XVL7, B09JDWSQT..."
3606246,"[B085JFX7MP, B085JGHW8R]",B01MPWVD44,IT,B085JGHW8R,"[B085JGHW8R, B07RYSKF2Z, B087S2CBGV, B08NVNBQN..."
3606247,"[B00B0UING2, B00B0UING2]",B00D3HYEZ4,IT,B00B0UING2,"[B00B0UING2, B00D3HYEZ4]"


In [17]:
# TODO: 不一定历史prev_items中的就不会买了，可以去掉限制试试！

In [18]:
train_preds = []
mrr = []

for _, row in tqdm(df_train.iterrows(), total=len(df_train)):
    label = row['next_item']
    pred_orig = row['next_item_prediction']
    pred = pred_orig
    prev_items = row['prev_items']
    if type(pred) == float:  # pred=nan, 训练集的last_item在之间没有统计出来下一跳，此时用最热门代替
        pred = top200[:100]
    else:
        if len(pred_orig) < 100:  # 小于100，从最热门选
            for i in top200:
                if i not in pred_orig and i not in prev_items:
                    pred.append(i)
                if len(pred) >= 100:
                    break
        else:
            pred = pred[:100]
    try:
        pred_result = pred.index(label)
        mrr.append(1 / (pred_result + 1))
    except:
        mrr.append(0)
    train_preds.append(pred)

np.mean(mrr)

100%|██████████| 3606249/3606249 [06:44<00:00, 8919.92it/s] 


0.21132176370327418

In [136]:
pickle.dump(train_preds, open('./data/train_preds_next_one_onlytask2_phase2.pkl', 'wb'))

In [116]:
df_train['next_item_prediction'] = df_train['last_item'].map(window_one_item_map)
df_train

,prev_items,next_item,locale,last_item,next_item_prediction
0,"[B09W9FND7K, B09JSPLN1M]",B09M7GY217,DE,B09JSPLN1M,"[B09W9FND7K, B09JSPLN1M, B098MKBJ2F, B09M7GY21..."
1,"[B076THCGSG, B007MO8IME, B08MF65MLV, B001B4TKA0]",B001B4THSA,DE,B001B4TKA0,"[B001B4THSA, B0001NPYZ2, B08MF65MLV, B01N3LKN0..."
2,"[B0B1LGXWDS, B00AZYORS2, B0B1LGXWDS, B00AZYORS...",B0767DTG2Q,DE,B00AZYORS2,"[B0B1LGXWDS, B083B866RY, B00AZYORS2, B0767DTG2..."
3,"[B09XMTWDVT, B0B4MZZ8MB, B0B7HZ2GWX, B09XMTWDV...",B0B4R9NN4B,DE,B0B71CHT1L,"[B09XMTWDVT, B0B7HZ2GWX, B0B71GSJ2R, B0B4MZZ8M..."
4,"[B09Y5CSL3T, B09Y5DPTXN, B09FKD61R8]",B0BGVBKWGZ,DE,B09FKD61R8,"[B09FKF5W9Q, B09Y5DPTXN, B0BGVBKWGZ, B09MRVWQ3..."
...,...,...,...,...,...
3606244,"[B086CYFSKW, B0874F9859, B086CYFSKW]",B07B5TYD76,IT,B086CYFSKW,"[B0874F9859, B08FJQFPLM, B07B5TYD76]"
3606245,"[B09NRZKZ7V, B08WJTPV93]",B08L1P4C3D,IT,B08WJTPV93,"[B08WJTPV93, B09NRZKZ7V, B088N7XVL7, 884722420..."
3606246,"[B085JFX7MP, B085JGHW8R]",B01MPWVD44,IT,B085JGHW8R,"[B085JGHW8R, B01MPWVD44, B087S2CBGV, B07RYSKF2..."
3606247,"[B00B0UING2, B00B0UING2]",B00D3HYEZ4,IT,B00B0UING2,"[B00B0UING2, B00D3HYEZ4, B00V4XSTUU, B09LCYDC4..."


In [117]:
train_preds = []
mrr = []

for _, row in tqdm(df_train.iterrows(), total=len(df_train)):
    label = row['next_item']
    pred_orig = row['next_item_prediction']
    pred = pred_orig
    prev_items = row['prev_items']
    if type(pred) == float:  # pred=nan, 训练集的last_item在之间没有统计出来下一跳，此时用最热门代替
        pred = top200_window_one[:100]
    else:
        if len(pred_orig) < 100:  # 小于100，从最热门选
            for i in top200_window_one:
                if i not in pred_orig and i not in prev_items:
                    pred.append(i)
                if len(pred) >= 100:
                    break
        else:
            pred = pred[:100]
    try:
        pred_result = pred.index(label)
        mrr.append(1 / (pred_result + 1))
    except:
        mrr.append(0)
    train_preds.append(pred)

np.mean(mrr)

100%|██████████| 3606249/3606249 [07:23<00:00, 8132.24it/s] 


0.3421572021672678

In [118]:
pickle.dump(train_preds, open('./data/train_preds_window_one_onlytask2_phase2.pkl', 'wb'))

测试集预测结果

In [19]:
next_item_dict = pickle.load(open('data/window_one_item_dict_onlytask2_phase2.pkl', 'rb'))
next_item_map = pickle.load(open('data/window_one_item_map_onlytask2_phase2.pkl', 'rb'))
top200 = pickle.load(open('data/top200_window_one_onlytask2_phase2.pkl', 'rb'))

In [20]:
df_test_phase2['next_item_prediction'] = df_test_phase2['last_item'].map(next_item_map)
df_test_phase2

,prev_items,locale,last_item,next_item_prediction
0,"[B07GTS7SWK, B07GTS7SWK]",ES,B07GTS7SWK,"[B07GTS7SWK, B074574XPT, B074581R8Y, B0745795K..."
1,"[B0B33YWVHR, 849988993X, B09QQG85HM, B0BJZLJXR...",ES,B09K7TDY1H,"[B09M8LNB61, B09K7TDY1H, B09J4T4JF5, B08BRGH7S..."
2,"[B08FMPXDTJ, B0B4612MTM, B0B45YR21M, B0B4F2L38...",ES,B0B45YR21M,"[B0B4612MTM, B0B461V1RB, B0B4614XL6, B0B4625TR..."
3,"[B07R3W4XQ7, B07R3FB5B5]",ES,B07R3FB5B5,"[B07R3W4XQ7, B07R4VG9X3, B07R4WKSSV, B081T3NK4..."
4,"[B09V4KBWPL, B09G9FTLPB]",ES,B09G9FTLPB,"[B09G9LF91K, B09G9DMQ7M, B09G99D95Q, B09G97SLG..."
...,...,...,...,...
34685,"[B089F3YK6X, B01BBW9TEI, B01BBW9Q46]",IT,B01BBW9Q46,"[B01BBW9TEI, B01F545TZQ, B01BBW9OAC, B09QH4W4L..."
34686,"[B078XYC42V, B09TW31SND]",IT,B09TW31SND,"[B08DYGN5W7, B00ECBKODQ, B00IBDVTSU, B09TW215N..."
34687,"[B07ZG3V3X2, B09PGKF9Y8]",IT,B09PGKF9Y8,"[B07ZH7HYDX, B00P2J6IP0, B09PGKF9Y8, B07KXXCDS..."
34688,"[B0B3JRGZLM, B09SHQJ8NV]",IT,B09SHQJ8NV,"[B0B2PFFCBH, B09RBZ3XDV, B0B3JRGZLM, B09Q9C9QP..."


In [21]:
test_preds = []

for _, row in tqdm(df_test_phase2.iterrows(), total=len(df_test_phase2)):
    pred_orig = row['next_item_prediction']
    pred = pred_orig
    prev_items = row['prev_items']
    if type(pred) == float:  # pred=nan, 测试集的last_item在之间没有统计出来下一跳，此时用最热门代替
        pred = top200[:100]
    else:
        if len(pred_orig) < 100:  # 小于100，从最热门选
            for i in top200:
                if i not in pred_orig and i not in prev_items:
                    pred.append(i)
                if len(pred) >= 100:
                    break
        else:
            pred = pred[:100]
    test_preds.append(pred)

100%|██████████| 34690/34690 [00:03<00:00, 9216.46it/s] 


In [22]:
len(test_preds)

34690

In [140]:
pickle.dump(test_preds, open('./data/test_preds_next_one_onlytask2_phase2.pkl', 'wb'))

In [18]:
# test_preds = pickle.load(open('./data/test_preds.pkl', 'rb'))

In [119]:
df_test_phase2['next_item_prediction'] = df_test_phase2['last_item'].map(window_one_item_map)
df_test_phase2

,prev_items,locale,last_item,next_item_prediction
0,"[B07GTS7SWK, B07GTS7SWK]",ES,B07GTS7SWK,"[B07GTS7SWK, B074574XPT, B074581R8Y, B0745795K..."
1,"[B0B33YWVHR, 849988993X, B09QQG85HM, B0BJZLJXR...",ES,B09K7TDY1H,"[B09M8LNB61, B09K7TDY1H, B09J4T4JF5, B08BRGH7S..."
2,"[B08FMPXDTJ, B0B4612MTM, B0B45YR21M, B0B4F2L38...",ES,B0B45YR21M,"[B0B4612MTM, B0B461V1RB, B0B4614XL6, B0B4625TR..."
3,"[B07R3W4XQ7, B07R3FB5B5]",ES,B07R3FB5B5,"[B07R3W4XQ7, B07R4VG9X3, B07R4WKSSV, B081T3NK4..."
4,"[B09V4KBWPL, B09G9FTLPB]",ES,B09G9FTLPB,"[B09G9LF91K, B09G9DMQ7M, B09G99D95Q, B09G97SLG..."
...,...,...,...,...
34685,"[B089F3YK6X, B01BBW9TEI, B01BBW9Q46]",IT,B01BBW9Q46,"[B01BBW9TEI, B01F545TZQ, B01BBW9OAC, B09QH4W4L..."
34686,"[B078XYC42V, B09TW31SND]",IT,B09TW31SND,"[B08DYGN5W7, B00ECBKODQ, B00IBDVTSU, B09TW215N..."
34687,"[B07ZG3V3X2, B09PGKF9Y8]",IT,B09PGKF9Y8,"[B07ZH7HYDX, B00P2J6IP0, B09PGKF9Y8, B07KXXCDS..."
34688,"[B0B3JRGZLM, B09SHQJ8NV]",IT,B09SHQJ8NV,"[B0B2PFFCBH, B09RBZ3XDV, B0B3JRGZLM, B09Q9C9QP..."


In [120]:
test_preds = []

for _, row in tqdm(df_test_phase2.iterrows(), total=len(df_test_phase2)):
    pred_orig = row['next_item_prediction']
    pred = pred_orig
    prev_items = row['prev_items']
    if type(pred) == float:  # pred=nan, 测试集的last_item在之间没有统计出来下一跳，此时用最热门代替
        pred = top200_window_one[:100]
    else:
        if len(pred_orig) < 100:  # 小于100，从最热门选
            for i in top200_window_one:
                if i not in pred_orig and i not in prev_items:
                    pred.append(i)
                if len(pred) >= 100:
                    break
        else:
            pred = pred[:100]
    test_preds.append(pred)

100%|██████████| 34690/34690 [00:03<00:00, 10648.92it/s]


In [121]:
pickle.dump(test_preds, open('./data/test_preds_window_one_onlytask2_phase2.pkl', 'wb'))

保存结果

In [23]:
df_test_phase2['next_item_prediction'] = test_preds
df_test_phase2['next_item_prediction'].apply(len).describe()

count    34690.0
mean       100.0
std          0.0
min        100.0
25%        100.0
50%        100.0
75%        100.0
max        100.0
Name: next_item_prediction, dtype: float64

In [24]:
df_test_phase2[['locale', 'next_item_prediction']]

,locale,next_item_prediction
0,ES,"[B07GTS7SWK, B074574XPT, B074581R8Y, B0745795K..."
1,ES,"[B09M8LNB61, B09K7TDY1H, B09J4T4JF5, B08BRGH7S..."
2,ES,"[B0B4612MTM, B0B461V1RB, B0B4614XL6, B0B4625TR..."
3,ES,"[B07R3W4XQ7, B07R4VG9X3, B07R4WKSSV, B081T3NK4..."
4,ES,"[B09G9LF91K, B09G9DMQ7M, B09G99D95Q, B09G97SLG..."
...,...,...
34685,IT,"[B01BBW9TEI, B01F545TZQ, B01BBW9OAC, B09QH4W4L..."
34686,IT,"[B08DYGN5W7, B00ECBKODQ, B00IBDVTSU, B09TW215N..."
34687,IT,"[B07ZH7HYDX, B00P2J6IP0, B09PGKF9Y8, B07KXXCDS..."
34688,IT,"[B0B2PFFCBH, B09RBZ3XDV, B0B3JRGZLM, B09Q9C9QP..."


In [142]:
df_test_phase2[['locale', 'next_item_prediction']].to_parquet('output/phase2_submission_task2_rule_next_one_zhaohui_onlytask2.parquet', engine='pyarrow')

In [36]:
df_test_phase2[['locale', 'next_item_prediction']].to_parquet('output/phase2_submission_task2_rule_window_one_onlytask2_testtest.parquet', engine='pyarrow')

In [88]:
df_test_phase2[['locale', 'next_item_prediction']]

,locale,next_item_prediction
0,ES,"[B07GTS7SWK, B074574XPT, B0745795KF, B074RZKWM..."
1,ES,"[B09M8LNB61, B09K7TDY1H, B09J4T4JF5, B09NT33LZ..."
2,ES,"[B0B4612MTM, B0B461V1RB, B0B4614XL6, B0B4625TR..."
3,ES,"[B07R3W4XQ7, B07R4VG9X3, B07R4WKSSV, B081T3NK4..."
4,ES,"[B09G9LF91K, B09G9DMQ7M, B09G99D95Q, B09G97SLG..."
...,...,...
34685,IT,"[B01BBW9TEI, B01F545TZQ, B09QH4W4LS, B01BBW9Q4..."
34686,IT,"[B08DYGN5W7, B00ECBKODQ, B00IBDVTSU, B08R57HZ4..."
34687,IT,"[B00P2J6IP0, B09PGKF9Y8, B07KXXCDS2, B07ZH7HYD..."
34688,IT,"[B09RBZ3XDV, B0B2PFFCBH, B09W2Z395Q, B0B3JRGZL..."


# 数据处理

产品表处理

In [21]:
# TODO: 部分ID不在产品表里？？没有！

In [17]:
products_ids = set(products.index.get_level_values('id'))

KeyError: 'Requested level (id) does not match index name (None)'

In [ ]:
len(products_ids)

1341314

In [135]:
total_ids = set(train_ids) | set(test_ids)

In [136]:
len(total_ids)

1338847

In [19]:
# products = products[products['locale'].isin(['DE', 'JP', 'UK'])]

In [54]:
products.shape 

(1551057, 11)

In [58]:
products.drop_duplicates(subset=['id'], keep='first', inplace=True)
products.shape 

(1410675, 11)

In [13]:
# products = products.set_index(['id', 'locale'])

In [59]:
products.head()

,id,locale,title,price,brand,color,size,model,material,author,desc
0,B005ZSSN10,DE,RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wo...,30.95,RED DRAGON,NaN,NaN,RDD0089,NaN,NaN,Amberjacks Steel Dartpfeile sind verfügbar in ...
1,B08PRYN6LD,DE,Simply Keto Lower Carb* Schokodrops ohne Zucke...,17.90,Simply Keto,NaN,750 g (1er Pack),NaN,NaN,NaN,🌱 NATÜRLICHE SÜSSE DURCH ERYTHRIT - Wir stelle...
2,B09MBZJ48V,DE,"Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...",68.89,Sennheiser,Multi-Colour,One size,508377,Kunstleder,NaN,3.5 MM BUCHSE - Kann problemlos an Geräte mit ...
3,B08ZN6F26S,DE,AmyBenton Auto ab 1 2 3 ahre - Baby Aufziehbar...,18.99,Amy & Benton,Animal Car,NaN,2008B,aufziehauto 1 jahr,NaN,【Auto aufziehbar】: Drücken Sie einfach leicht ...
4,B094DGRV7D,DE,PLAYMOBIL - 70522 - Cavaliere mit grauem Pony,7.17,PLAYMOBIL,Nicht Zutreffend.,OneSize,70522,Polypropylen,NaN,Inhalt: 1 Stück


In [60]:
# products.loc[('B005ZSSN10', 'DE')]

In [61]:
products['len_title'] = products['title'].apply(lambda x : 0 if type(x) == float else len(x))
products['len_desc'] = products['desc'].apply(lambda x : 0 if type(x) == float else len(x))

In [62]:
products.nunique()

id           1410675
locale             6
title        1360094
price          37608
brand         177189
color         203260
size          218060
model         524101
material       45568
author         30835
desc          800044
len_title        480
len_desc         763
dtype: int64

In [63]:
products.columns

Index(['id', 'locale', 'title', 'price', 'brand', 'color', 'size', 'model',
       'material', 'author', 'desc', 'len_title', 'len_desc'],
      dtype='object')

In [64]:
cat_features = ['brand', 'color', 'size', 'model', 'material', 'author']
num_features = ['price', 'len_title', 'len_desc']

In [65]:
for f in cat_features:
    le = LabelEncoder()
    products[f] = le.fit_transform(products[f])

In [66]:
products.shape 

(1410675, 13)

In [67]:
products.head()

,id,locale,title,price,brand,color,size,model,material,author,desc,len_title,len_desc
0,B005ZSSN10,DE,RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wo...,30.95,112134,203260,218060,426630,45568,30835,Amberjacks Steel Dartpfeile sind verfügbar in ...,96,121
1,B08PRYN6LD,DE,Simply Keto Lower Carb* Schokodrops ohne Zucke...,17.90,124505,203260,128007,524101,45568,30835,🌱 NATÜRLICHE SÜSSE DURCH ERYTHRIT - Wir stelle...,186,330
2,B09MBZJ48V,DE,"Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...",68.89,122979,114264,170270,145013,15566,30835,3.5 MM BUCHSE - Kann problemlos an Geräte mit ...,181,95
3,B08ZN6F26S,DE,AmyBenton Auto ab 1 2 3 ahre - Baby Aufziehbar...,18.99,9834,46931,218060,67408,29357,30835,【Auto aufziehbar】: Drücken Sie einfach leicht ...,101,191
4,B094DGRV7D,DE,PLAYMOBIL - 70522 - Cavaliere mit grauem Pony,7.17,105135,117844,170305,174527,23064,30835,Inhalt: 1 Stück,45,15


测试集跟产品表连表

In [78]:
df_test.columns

Index(['prev_items', 'locale', 'last_item'], dtype='object')

In [79]:
df_test_exploded = df_test[['prev_items', 'locale', 'last_item']].explode('prev_items')

In [80]:
df_test_exploded.index 

Int64Index([     0,      0,      0,      0,      0,      0,      0,      0,
                 0,      0,
            ...
            316968, 316968, 316968, 316968, 316968, 316969, 316969, 316970,
            316970, 316970],
           dtype='int64', length=1351132)

In [81]:
df_test_exploded['index'] = df_test_exploded.index 

In [82]:
df_test_exploded = df_test_exploded.rename(columns={'prev_items': 'id'})

In [83]:
df_test_exploded

,id,locale,last_item,index
0,B08V12CT4C,DE,B099NQFMG7,0
0,B08V1KXBQD,DE,B099NQFMG7,0
0,B01BVG1XJS,DE,B099NQFMG7,0
0,B09VC5PKN5,DE,B099NQFMG7,0
0,B09V7KG931,DE,B099NQFMG7,0
...,...,...,...,...
316969,B01MCQMORK,UK,B09JYZ325W,316969
316969,B09JYZ325W,UK,B09JYZ325W,316969
316970,B0B8JX92YJ,UK,B0BG2LZQSL,316970
316970,B09TN4MP6V,UK,B0BG2LZQSL,316970


In [84]:
# products在同一个地区的ID是不一样的，但是不同地区的ID可能会一样
df_test_exploded_fea = df_test_exploded.merge(products, on='id', how='left')
df_test_exploded_fea.head()

,id,locale_x,last_item,index,locale_y,title,price,brand,color,size,model,material,author,desc,len_title,len_desc
0,B08V12CT4C,DE,B099NQFMG7,0,DE,ThermoPro TP49 digitales Mini Thermo-Hygromete...,22.99,133388,203260,218060,524101,45568,30835,"Energiesparende Batterie: 1x 1,5 V AAA Batteri...",149,210
1,B08V1KXBQD,DE,B099NQFMG7,0,DE,ThermoPro TP49W-3 digitales Mini Thermo-Hygrom...,29.99,133388,203260,218060,524101,45568,30835,"Energiesparende Batterie: 1x 1,5 V AAA Batteri...",151,210
2,B01BVG1XJS,DE,B099NQFMG7,0,DE,Bresser Thermometer Hygrometer Temeo Hygro Ind...,19.95,22590,155573,93148,172859,15665,30835,"Abmessungen: 45 x 58 x 13 mm, Gewicht: 31 g; F...",170,139
3,B09VC5PKN5,DE,B099NQFMG7,0,DE,ThermoPro TP157 Mini Hygrometer Thermometer In...,12.99,133388,203260,218060,468395,45568,30835,Batterielaufzeit für 18 Monate: Die im Lieferu...,184,192
4,B09V7KG931,DE,B099NQFMG7,0,DE,ThermoPro TP157 Mini Hygrometer Thermometer In...,29.99,133388,203260,218060,468396,45568,30835,Batterielaufzeit für 18 Monate: Die im Lieferu...,192,192


In [85]:
df_test_exploded_fea.shape 

(1351132, 16)

In [86]:
df_test_exploded_fea[df_test_exploded_fea['index'] == 104606]

,id,locale_x,last_item,index,locale_y,title,price,brand,color,size,model,material,author,desc,len_title,len_desc
450255,B082T6R1HC,JP,B082T5WH22,104606,DE,Amazon Basics – Verbindungskabel Lightning auf...,40000000.07,9445,155573,2947,363144,45568,30835,NaN,135,0
450256,B082T6GVG9,JP,B082T5WH22,104606,DE,Amazon Basics – Verbindungskabel Lightning auf...,11.12,9442,155573,2947,363138,45568,30835,Lieferumfang: Verbindungskabel Lightning auf U...,106,139
450257,B082T5WH22,JP,B082T5WH22,104606,DE,Amazon Basics – Verbindungskabel Lightning auf...,11.99,9445,136603,8414,363145,45568,30835,Lieferumfang: Fortschrittliches Verbindungskab...,138,157


In [87]:
products.head() 

,id,locale,title,price,brand,color,size,model,material,author,desc,len_title,len_desc
0,B005ZSSN10,DE,RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wo...,30.95,112134,203260,218060,426630,45568,30835,Amberjacks Steel Dartpfeile sind verfügbar in ...,96,121
1,B08PRYN6LD,DE,Simply Keto Lower Carb* Schokodrops ohne Zucke...,17.90,124505,203260,128007,524101,45568,30835,🌱 NATÜRLICHE SÜSSE DURCH ERYTHRIT - Wir stelle...,186,330
2,B09MBZJ48V,DE,"Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...",68.89,122979,114264,170270,145013,15566,30835,3.5 MM BUCHSE - Kann problemlos an Geräte mit ...,181,95
3,B08ZN6F26S,DE,AmyBenton Auto ab 1 2 3 ahre - Baby Aufziehbar...,18.99,9834,46931,218060,67408,29357,30835,【Auto aufziehbar】: Drücken Sie einfach leicht ...,101,191
4,B094DGRV7D,DE,PLAYMOBIL - 70522 - Cavaliere mit grauem Pony,7.17,105135,117844,170305,174527,23064,30835,Inhalt: 1 Stück,45,15


训练集和产品表连表

In [88]:
df_train.columns

Index(['prev_items', 'next_item', 'locale', 'last_item'], dtype='object')

In [89]:
df_train_exploded = df_train[['prev_items', 'locale', 'last_item', 'next_item']].explode('prev_items')

In [90]:
df_train_exploded['index'] = df_train_exploded.index 

In [91]:
df_train_exploded = df_train_exploded.rename(columns={'prev_items': 'id'})

In [92]:
# products在同一个地区的ID是不一样的，但是不同地区的ID可能会一样
df_train_exploded_fea = df_train_exploded.merge(products, on='id', how='left')
df_train_exploded_fea.head()

,id,locale_x,last_item,next_item,index,locale_y,title,price,brand,color,size,model,material,author,desc,len_title,len_desc
0,B09W9FND7K,DE,B09JSPLN1M,B09M7GY217,0,DE,OREiN LED Einbaustrahler 230V Dimmbar & IP65 L...,39.99,101601,37318,163936,524101,39693,30835,【Angenehmes Licht】Dieser Einbaustrahler LED Fl...,196,222
1,B09JSPLN1M,DE,B09JSPLN1M,B09M7GY217,0,DE,OREiN LED Spot LED Einbaustrahler 230V IP65 LE...,36.99,101601,37319,163936,524101,15665,30835,【Angenehmes Licht】Dieser Einbaustrahler LED Fl...,199,231
2,B076THCGSG,DE,B001B4TKA0,B001B4THSA,1,DE,"VIETSCHI Profi DIN Plus 12,5L weiß - doppeldec...",39.80,139845,203260,23154,524101,45568,30835,"✅ Einfache Verarbeitung - Egal ob Streichen, R...",195,192
3,B007MO8IME,DE,B001B4TKA0,B001B4THSA,1,DE,"Caparol Capa DIN 12,500 L Wandfarben",40.27,27926,155413,23154,38821,10947,30835,Caparol,36,7
4,B08MF65MLV,DE,B001B4TKA0,B001B4THSA,1,DE,Metzler Edelstahl Türklingel in Anthrazit - Tü...,29.99,93325,47134,130654,524101,11437,30835,Der Anschluss erfolgt an herkömmlichen Klingel...,174,222


In [93]:
df_train_exploded_fea.shape 

(15306183, 17)

测试集序列特征抽取

In [94]:
agg = {
    'id': ['nunique', 'count'],
    'brand' : ['nunique', 'count'],
    'color' : ['nunique', 'count'],
    'size' : ['nunique', 'count'],
    'model' : ['nunique', 'count'],
    'material' : ['nunique', 'count'],
    'author' : ['nunique', 'count'],
    'price' : ['mean', 'std', 'min', 'max', 'sum'],
    'len_title' : ['mean', 'std', 'min', 'max', 'sum'],
    'len_desc' : ['mean', 'std', 'min', 'max', 'sum'],
}

In [95]:
df_test_seqs_feas = df_test_exploded_fea.groupby('index').agg(agg)

In [96]:
df_test_seqs_feas.columns = pd.Index([e[0] + e[1].upper() for e in df_test_seqs_feas.columns.tolist()])

In [97]:
df_test_seqs_feas.head()

,idNUNIQUE,idCOUNT,brandNUNIQUE,brandCOUNT,colorNUNIQUE,colorCOUNT,sizeNUNIQUE,sizeCOUNT,modelNUNIQUE,modelCOUNT,...,len_titleMEAN,len_titleSTD,len_titleMIN,len_titleMAX,len_titleSUM,len_descMEAN,len_descSTD,len_descMIN,len_descMAX,len_descSUM
index,,,,,,,,,,,,,,,,,,,,,
0,9,12,3,12,5,12,2,12,6,12,...,163.333333,24.436684,126,195,1960,173.583333,34.057327,120,210,2083
1,2,3,1,3,1,3,2,3,1,3,...,79.666667,2.886751,78,83,239,44.000000,76.210236,0,132,132
2,3,4,2,4,3,4,1,4,3,4,...,141.250000,63.641574,46,179,565,137.750000,79.197538,26,213,551
3,11,20,4,20,6,20,1,20,2,20,...,123.250000,65.841017,20,194,2465,0.000000,0.000000,0,0,0
4,7,7,3,7,6,7,3,7,5,7,...,159.571429,46.396326,110,200,1117,337.285714,17.279220,324,372,2361


In [98]:
df_test_seqs_feas.shape 

(316971, 29)

In [111]:
df_test_seqs_feas.to_csv('data/df_test_seqs_feas_all.csv', index=False)

训练集序列特征抽取

In [100]:
df_train_seqs_feas = df_train_exploded_fea.groupby('index').agg(agg)

In [101]:
df_train_seqs_feas.columns = pd.Index([e[0] + e[1].upper() for e in df_train_seqs_feas.columns.tolist()])

In [102]:
df_train_seqs_feas.head()

,idNUNIQUE,idCOUNT,brandNUNIQUE,brandCOUNT,colorNUNIQUE,colorCOUNT,sizeNUNIQUE,sizeCOUNT,modelNUNIQUE,modelCOUNT,...,len_titleMEAN,len_titleSTD,len_titleMIN,len_titleMAX,len_titleSUM,len_descMEAN,len_descSTD,len_descMIN,len_descMAX,len_descSUM
index,,,,,,,,,,,,,,,,,,,,,
0,2,2,1,2,2,2,1,2,1,2,...,197.500000,2.121320,196,199,395,226.500000,6.363961,222,231,453
1,4,4,4,4,4,4,3,4,3,4,...,122.000000,75.166482,36,195,488,126.750000,98.878292,7,222,507
2,2,9,1,9,1,9,1,9,1,9,...,121.111111,29.514591,90,146,1090,24.000000,0.000000,24,24,216
3,4,7,1,7,4,7,1,7,4,7,...,196.428571,4.466809,191,200,1375,348.714286,36.926568,314,409,2441
4,3,3,2,3,2,3,3,3,2,3,...,174.333333,31.785741,138,197,523,279.666667,120.088856,141,349,839


In [107]:
df_train_seqs_feas.shape

(3606249, 29)

In [110]:
df_train_seqs_feas.to_csv('data/df_train_seqs_feas_all.csv', index=False)

In [103]:
df_train_seqs_feas.columns

Index(['idNUNIQUE', 'idCOUNT', 'brandNUNIQUE', 'brandCOUNT', 'colorNUNIQUE',
       'colorCOUNT', 'sizeNUNIQUE', 'sizeCOUNT', 'modelNUNIQUE', 'modelCOUNT',
       'materialNUNIQUE', 'materialCOUNT', 'authorNUNIQUE', 'authorCOUNT',
       'priceMEAN', 'priceSTD', 'priceMIN', 'priceMAX', 'priceSUM',
       'len_titleMEAN', 'len_titleSTD', 'len_titleMIN', 'len_titleMAX',
       'len_titleSUM', 'len_descMEAN', 'len_descSTD', 'len_descMIN',
       'len_descMAX', 'len_descSUM'],
      dtype='object')

In [104]:
seqs_cat_feas = [f for f in df_train_seqs_feas.columns if 'NUNIQUE' in f or 'COUNT' in f]

In [105]:
seqs_num_feas = [f for f in df_train_seqs_feas.columns if f not in seqs_cat_feas]

In [106]:
df_train_seqs_feas[seqs_num_feas].isna().any()

priceMEAN        False
priceSTD         False
priceMIN         False
priceMAX         False
priceSUM         False
len_titleMEAN    False
len_titleSTD     False
len_titleMIN     False
len_titleMAX     False
len_titleSUM     False
len_descMEAN     False
len_descSTD      False
len_descMIN      False
len_descMAX      False
len_descSUM      False
dtype: bool

In [58]:
tmp = pd.concat([df_train_seqs_feas[seqs_cat_feas], df_test_seqs_feas[seqs_cat_feas]])

In [59]:
tmp.shape 

(3923220, 14)

In [61]:
tmp.nunique().to_dict()

{'idNUNIQUE': 113,
 'idCOUNT': 135,
 'brandNUNIQUE': 37,
 'brandCOUNT': 118,
 'colorNUNIQUE': 53,
 'colorCOUNT': 110,
 'sizeNUNIQUE': 60,
 'sizeCOUNT': 106,
 'modelNUNIQUE': 51,
 'modelCOUNT': 92,
 'materialNUNIQUE': 22,
 'materialCOUNT': 102,
 'authorNUNIQUE': 45,
 'authorCOUNT': 122}

In [72]:
(tmp.max() + 1).to_dict()

{'idNUNIQUE': 133,
 'idCOUNT': 475,
 'brandNUNIQUE': 39,
 'brandCOUNT': 475,
 'colorNUNIQUE': 53,
 'colorCOUNT': 269,
 'sizeNUNIQUE': 115,
 'sizeCOUNT': 475,
 'modelNUNIQUE': 59,
 'modelCOUNT': 475,
 'materialNUNIQUE': 25,
 'materialCOUNT': 475,
 'authorNUNIQUE': 65,
 'authorCOUNT': 190}

In [67]:
df_train_seqs_feas[seqs_cat_feas].max()

idNUNIQUE          132
idCOUNT            474
brandNUNIQUE        38
brandCOUNT         474
colorNUNIQUE        52
colorCOUNT         268
sizeNUNIQUE        114
sizeCOUNT          474
modelNUNIQUE        58
modelCOUNT         474
materialNUNIQUE     24
materialCOUNT      474
authorNUNIQUE       64
authorCOUNT        189
dtype: int64

In [69]:
df_test_seqs_feas[seqs_cat_feas].max()

idNUNIQUE           80
idCOUNT            112
brandNUNIQUE        33
brandCOUNT         112
colorNUNIQUE        38
colorCOUNT         112
sizeNUNIQUE         22
sizeCOUNT          112
modelNUNIQUE        41
modelCOUNT          66
materialNUNIQUE     12
materialCOUNT       70
authorNUNIQUE       28
authorCOUNT        109
dtype: int64

In [62]:
df_train_seqs_feas[seqs_cat_feas].nunique()

idNUNIQUE          113
idCOUNT            134
brandNUNIQUE        37
brandCOUNT         117
colorNUNIQUE        53
colorCOUNT         109
sizeNUNIQUE         60
sizeCOUNT          105
modelNUNIQUE        51
modelCOUNT          92
materialNUNIQUE     22
materialCOUNT      102
authorNUNIQUE       45
authorCOUNT        121
dtype: int64

In [56]:
df_test_seqs_feas[seqs_cat_feas].nunique()

idNUNIQUE          47
idCOUNT            63
brandNUNIQUE       28
brandCOUNT         61
colorNUNIQUE       30
colorCOUNT         58
sizeNUNIQUE        22
sizeCOUNT          53
modelNUNIQUE       30
modelCOUNT         49
materialNUNIQUE    13
materialCOUNT      50
authorNUNIQUE      21
authorCOUNT        37
dtype: int64

In [64]:
df_train_seqs_feas[seqs_num_feas].head()

,priceMEAN,priceSTD,priceMIN,priceMAX,priceSUM,len_titleMEAN,len_titleSTD,len_titleMIN,len_titleMAX,len_titleSUM,len_descMEAN,len_descSTD,len_descMIN,len_descMAX,len_descSUM
index,,,,,,,,,,,,,,,
0,38.490000,2.121320,36.99,39.99,76.98,197.500000,2.121320,196.0,199.0,395.0,226.500000,6.363961,222.0,231.0,453.0
1,31.762500,10.929280,16.99,40.27,127.05,122.000000,75.166482,36.0,195.0,488.0,126.750000,98.878292,7.0,222.0,507.0
2,60.873333,8.427470,51.99,67.98,547.86,121.111111,29.514591,90.0,146.0,1090.0,24.000000,0.000000,24.0,24.0,216.0
3,14.418571,0.534522,13.99,14.99,100.93,196.428571,4.466809,191.0,200.0,1375.0,348.714286,36.926568,314.0,409.0,2441.0
4,23.656667,11.590226,12.99,35.99,70.97,174.333333,31.785741,138.0,197.0,523.0,279.666667,120.088856,141.0,349.0,839.0


In [57]:
df_test_seqs_feas[seqs_num_feas].values.shape

(316971, 15)

In [65]:
# df_train_seqs_feas.to_csv('data/df_train_seqs_feas_all.csv', index=False)

In [55]:
products

,,title,price,brand,color,size,model,material,author,desc,len_title,len_desc
id,locale,,,,,,,,,,,
B005ZSSN10,DE,RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wo...,30.95,105804,199923,213319,399670,44782,29259,Amberjacks Steel Dartpfeile sind verfügbar in ...,96,121
B08PRYN6LD,DE,Simply Keto Lower Carb* Schokodrops ohne Zucke...,17.90,117662,199923,124815,492997,44782,29259,🌱 NATÜRLICHE SÜSSE DURCH ERYTHRIT - Wir stelle...,186,330
B09MBZJ48V,DE,"Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...",68.89,116175,112280,166049,135239,15196,29259,3.5 MM BUCHSE - Kann problemlos an Geräte mit ...,181,95
B08ZN6F26S,DE,AmyBenton Auto ab 1 2 3 ahre - Baby Aufziehbar...,18.99,9251,46812,213319,62805,28665,29259,【Auto aufziehbar】: Drücken Sie einfach leicht ...,101,191
B094DGRV7D,DE,PLAYMOBIL - 70522 - Cavaliere mit grauem Pony,7.17,99190,115345,166088,162740,22373,29259,Inhalt: 1 Stück,45,15
...,...,...,...,...,...,...,...,...,...,...,...,...
B08D7KW8VK,UK,TOMHOUSEE Anime Cosplay Short Straight Hair Wi...,9.99,123307,75921,213319,492997,26052,29259,NaN,124,0
B073WXLXR9,UK,Crystals NEW brilliant ink twister bingo dabbe...,8.99,25194,118197,213319,492997,20933,29259,NaN,85,0
1529393833,UK,"Before I Do: the new, funny and unexpected lov...",4.50,61247,199923,213319,492997,44782,6063,NaN,92,0


In [56]:
products.columns

Index(['title', 'price', 'brand', 'color', 'size', 'model', 'material',
       'author', 'desc', 'len_title', 'len_desc'],
      dtype='object')

In [57]:
products_use = products[['price', 'brand', 'color', 'size', 'model', 'material', 'author', 'len_title', 'len_desc']]

In [58]:
products_use.loc[('B005ZSSN10', 'DE')].values.tolist()

[30.95, 105804.0, 199923.0, 213319.0, 399670.0, 44782.0, 29259.0, 96.0, 121.0]

# 排序

In [63]:
# def get_feature(index, train_seq, seq_featurs_index):
#     train_fea = []

#     label = train_seq['next_item'].iloc[0]
#     last_item = df_train['last_item'].iloc[0]
#     if last_item in next_item_map:
#         next_item_prediction = next_item_map[last_item]
#     else:
#         next_item_prediction = []
#     # print(next_item_prediction)
#     prev_items = list(train_seq['id'])
#     locale = train_seq['locale'].iloc[0]
#     loc_id = loc2id[locale]  # 地区ID

#     candidate_id = []
#     candidate_id = next_item_prediction  # 1-hop预测的结果
#     if len(next_item_prediction) == 0:  # pred=nan, 训练集的last_item在之间没有统计出来下一跳，此时用最热门代替
#         candidate_id = top200[:100]
#     else:
#         if len(candidate_id) < 100:  # 小于100，从最热门选
#             for i in top200:
#                 if i not in candidate_id and i not in prev_items:
#                     candidate_id.append(i)
#                 if len(candidate_id) >= 100:
#                     break
#         else:
#             candidate_id = candidate_id[:100]
#     # 其他召回策略，比如hop200, w2v等
#     # 另外，这个位置的写法之后改成芒果TV的做法，比如nan的时候直接top200也可能会有历史重复的物品，这样不太好！！！
#     # 目前先验证这个固定的召回+排序之后会不会编号
    
#     # 构造一些序列特征 根据历史的seq，就是seq_featurs_index
    
#     # TODO: 加入item本身的特征 通过连表操作
#     # TODO: 候选集太多，数据量太大了，sample 3个负样本算了
#     if label not in candidate_id:
#         sample = candidate_id[:5]
#         #sample = np.random.choice(candidate_id, size=3, replace=False).tolist()
#     else:
#         candidate_id.remove(label)
#         sample = candidate_id[:5]
#         #sample = np.random.choice(candidate_id, size=3, replace=False).tolist()
#     sample.append(label)
#     for vid in sample:
#         if vid == label:
#             lbl = 1
#         else:
#             lbl = 0
#         # print(vid, locale)
#         try:
#             tmp = products.loc[(vid, locale)]  # 产品特征
#             item_price = tmp.price
#             item_brand = tmp.brand 
#             item_color = tmp.color 
#             item_size = tmp.size 
#             item_model = tmp.model 
#             item_material = tmp.material 
#             item_author = tmp.author 
#         except:
#             continue
        
#         feat = [
#             # index,  这个比赛没有用户ID这一个特征！
#             int(vid in prev_items),  # 候选的vid是否出现在历史购买的序列中
#             int(vid in next_item_prediction),  # 候选的vid是否出现在上次购买产品的下一跳序列中
#             loc_id,
#             item_price,
#             item_brand,
#             item_color,
#             item_size,
#             item_model,
#             item_material,
#             item_author,
#             # 其他特征
#         ]
#         feat += seq_featurs_index
#         # try:
#         #     feat.append(vid_w2v.wv.similarity(vid, last1_vid))
#         # except:
#         #     feat.append(0)
#         feat.append(lbl)
#         train_fea.append(feat)
#     return train_fea

In [64]:
# a = get_feature(0, df_train_exploded_fea[:2], df_train_seqs_feas.loc[0].values.tolist())

训练特征

In [59]:
def get_train_feature(index, train_seq, seq_featurs_index, products_feas):
    train_fea = []

    label = train_seq['next_item'].iloc[0]
    prev_items = list(train_seq['id'])
    locale = train_seq['locale'].iloc[0]
    loc_id = loc2id[locale]  # 地区ID

    candidate_id = train_preds[index]
    # 其他召回策略，比如hop200, w2v等
    # 另外，这个位置的写法之后改成芒果TV的做法，比如nan的时候直接top200也可能会有历史重复的物品，这样不太好！！！
    # 目前先验证这个固定的召回+排序之后会不会编号
    
    # 构造一些序列特征 根据历史的seq，就是seq_featurs_index
    
    # TODO: 候选集太多，数据量太大了，sample 5个负样本算了
    if label in candidate_id:
        candidate_id.remove(label)
    
    # 5个负样本
    cnt = 0
    for vid in candidate_id:
        try:
            pro_feas_index = products_feas.loc[(vid, locale)].values.tolist()  # 产品特征
            cnt += 1
            if cnt > 5:
                break
        except:
            continue
        
        feat = [
            # index,  这个比赛没有用户ID这一个特征！
            int(vid in prev_items),  # 候选的vid是否出现在历史购买的序列中
            loc_id,
            # 其他特征
        ]
        feat += pro_feas_index
        feat += seq_featurs_index
        # try:
        #     feat.append(vid_w2v.wv.similarity(vid, last1_vid))
        # except:
        #     feat.append(0)
        feat.append(0)  # label
        train_fea.append(feat)
    
    # 正样本
    vid = label
    try:
        pro_feas_index = products_feas.loc[(vid, locale)].values.tolist()  # 产品特征
    except:
        print(vid, locale)
        
    feat = [
        # TODO: 重新跑这个训练特征
        vid,
        index,  # 这个比赛没有用户ID这一个特征！
        int(vid in prev_items),  # 候选的vid是否出现在历史购买的序列中
        loc_id,
        # 其他特征
    ]
    feat += pro_feas_index
    feat += seq_featurs_index
    # try:
    #     feat.append(vid_w2v.wv.similarity(vid, last1_vid))
    # except:
    #     feat.append(0)
    feat.append(1)  # label
    train_fea.append(feat)
    return train_fea

In [74]:
# train_fea = []
# df_train_len = df_train_exploded_fea['index'].nunique()
# for ind, train_seq_index in tqdm(df_train_exploded_fea.groupby('index'), total=df_train_len):
#     train_fea_in = get_train_feature(ind, train_seq_index, df_train_seqs_feas.loc[ind].values.tolist(), products_feas=products_use)
#     train_fea += train_fea_in

100%|██████████| 3272716/3272716 [6:13:14<00:00, 146.14it/s]   


In [188]:
train_columns = ['vid_in_his', 'loc_id'] + list(products_use.columns) + list(df_train_seqs_feas.columns) + ['label']

In [182]:
len(train_columns)

41

In [75]:
np.save('./data/train_features.npy', train_fea)

In [175]:
train_fea = np.load('./data/train_features.npy')

In [176]:
train_fea.shape 

(19636288, 41)

In [183]:
train_fea = pd.DataFrame(train_fea, columns=train_columns)
train_fea = train_fea.fillna(0)
print(train_fea.iloc[:, -1].value_counts(normalize=True))

0.0    0.833333
1.0    0.166667
Name: label, dtype: float64


In [217]:
for col in train_fea.columns:
    if col in cat_features:
        train_fea[col] = train_fea[col].astype('int64')
    else:
        train_fea[col] = train_fea[col].astype('float64')

In [218]:
train_fea.head()

,vid_in_his,loc_id,price,brand,color,size,model,material,author,len_title,...,len_titleMIN,len_titleMAX,len_titleSUM,len_descMEAN,len_descSTD,len_descMIN,len_descMAX,len_descSUM,label,prob
0,1.0,0.0,2.12132,196,199,395,226,6,222,231.0,...,1.0,2.0,38.49,2.12132,36.99,39.99,76.98,197.5,0.0,0.000007
1,0.0,0.0,2.12132,196,199,395,226,6,222,231.0,...,1.0,2.0,38.49,2.12132,36.99,39.99,76.98,197.5,0.0,0.215880
2,0.0,0.0,2.12132,196,199,395,226,6,222,231.0,...,1.0,2.0,38.49,2.12132,36.99,39.99,76.98,197.5,0.0,0.271886
3,0.0,0.0,2.12132,196,199,395,226,6,222,231.0,...,1.0,2.0,38.49,2.12132,36.99,39.99,76.98,197.5,0.0,0.259654
4,0.0,0.0,2.12132,196,199,395,226,6,222,231.0,...,1.0,2.0,38.49,2.12132,36.99,39.99,76.98,197.5,0.0,0.264382


测试特征

In [83]:
def get_test_feature(index, test_seq, seq_featurs_index, products_feas):
    test_fea = []

    prev_items = list(test_seq['id'])
    locale = test_seq['locale'].iloc[0]
    loc_id = loc2id[locale]  # 地区ID

    candidate_id = test_preds[index]
    # 其他召回策略，比如hop200, w2v等
    # 另外，这个位置的写法之后改成芒果TV的做法，比如nan的时候直接top200也可能会有历史重复的物品，这样不太好！！！
    # 目前先验证这个固定的召回+排序之后会不会编号
    
    # 构造一些序列特征 根据历史的seq，就是seq_featurs_index
    
    cnt = 0
    for vid in candidate_id:
        try:
            pro_feas_index = products_feas.loc[(vid, locale)].values.tolist()  # 产品特征
            cnt += 1
            if cnt > 10:
                break
        except:
            continue  # TODO: continue的话，概率怎么跟vid对应
        
        feat = [
            vid,
            index,  # 这个比赛没有用户ID这一个特征！
            int(vid in prev_items),  # 候选的vid是否出现在历史购买的序列中
            loc_id,
            # 其他特征
        ]
        feat += pro_feas_index
        feat += seq_featurs_index
        # try:
        #     feat.append(vid_w2v.wv.similarity(vid, last1_vid))
        # except:
        #     feat.append(0)
        # feat.append(0)  # label
        test_fea.append(feat)
    return test_fea

In [86]:
test_fea = []
df_test_len = df_test_exploded_fea['index'].nunique()
for ind, test_seq_index in tqdm(df_test_exploded_fea.groupby('index'), total=df_test_len):
    test_fea_in = get_test_feature(ind, test_seq_index, df_test_seqs_feas.loc[ind].values.tolist(), products_feas=products_use)
    test_fea += test_fea_in
    if len(test_fea_in) != 10:
        print(ind) 

 43%|████▎     | 135110/316971 [27:36<28:56, 104.71it/s] 

135090


 55%|█████▌    | 175760/316971 [32:16<24:19, 96.73it/s] 

175732


 56%|█████▌    | 177906/316971 [32:31<42:52, 54.07it/s] 

...

 61%|██████    | 193042/316971 [34:13<18:51, 109.53it/s] 

193020


 72%|███████▏  | 227523/316971 [42:40<15:56, 93.50it/s]   

227523


100%|██████████| 316971/316971 [1:09:40<00:00, 75.82it/s]


In [187]:
test_columns = ['vid', 'index', 'vid_in_his', 'loc_id'] + list(products_use.columns) + list(df_train_seqs_feas.columns)

In [189]:
len(test_columns)

42

In [114]:
np.save('./data/test_features_10_withID.npy', test_fea.values)

In [192]:
test_fea = np.load('./data/test_features_10_withID.npy', allow_pickle=True)

In [193]:
test_fea = pd.DataFrame(test_fea, columns=test_columns)
test_fea = test_fea.fillna(0)

In [194]:
test_fea.shape 

(3169697, 42)

In [213]:
test_fea['index'] = test_fea['index'].astype('int64')
for col in test_fea.columns[2:]:
    if col in cat_features:
        test_fea[col] = test_fea[col].astype('int64')
    else:
        test_fea[col] = test_fea[col].astype('float64')

In [201]:
test_fea.head()

,vid,index,vid_in_his,loc_id,price,brand,color,size,model,material,...,len_titleMEAN,len_titleSTD,len_titleMIN,len_titleMAX,len_titleSUM,len_descMEAN,len_descSTD,len_descMIN,len_descMAX,len_descSUM
0,B099NS1XPG,0,0.0,0.0,39.99,21236,152211,213319,161238,44782,...,163.333333,24.436684,126.0,195.0,1960.0,173.583333,34.057327,120.0,210.0,2083.0
1,B08496TCCQ,0,1.0,0.0,40000000.07,21236,133489,213319,161226,44782,...,163.333333,24.436684,126.0,195.0,1960.0,173.583333,34.057327,120.0,210.0,2083.0
2,B01BVG1XJS,0,1.0,0.0,19.95,21236,152211,90884,161227,15295,...,163.333333,24.436684,126.0,195.0,1960.0,173.583333,34.057327,120.0,210.0,2083.0
3,B099NR3X6D,0,0.0,0.0,32.99,21236,133489,213319,161237,44782,...,163.333333,24.436684,126.0,195.0,1960.0,173.583333,34.057327,120.0,210.0,2083.0
4,B09WDSH4CD,0,0.0,0.0,24.99,131540,152211,213319,492997,44782,...,163.333333,24.436684,126.0,195.0,1960.0,173.583333,34.057327,120.0,210.0,2083.0


lgb

In [68]:
seed = 2023
kfold = 5

In [69]:
lgb_params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    # 'metric': 'binary_logloss',  # auc??
    'metric': 'auc',
    'learning_rate': 0.05,
    'num_leaves': 2 ** 6,
    'max_depth': 8,
    'tree_learner': 'serial',
    'colsample_bytree': 0.8,
    'subsample_freq': 1,
    'subsample': 0.8,
    'num_boost_round': 3000,
    'max_bin': 255,
    'verbose': -1,
    'nthread' : -1,
    'seed': seed,
    'bagging_seed': seed,
    'feature_fraction_seed': seed,
    'early_stopping_rounds': 100,
    # 'device': 'gpu',  # 设置使用 GPU 加速
    # 'gpu_platform_id': 0,  # 设置 GPU 平台 id
    # 'gpu_device_id': 0  # 设置 GPU 设备 id
}

In [ ]:
# TODO: 训练集最后一列去掉是label，测试集第一列去掉是vid

In [202]:
if 'prob' in train_fea.columns:
    train_fea = train_fea.drop('prob', axis=1)
train = train_fea.iloc[:, 0: -1]
if 'prob' in test_fea.columns:
    test_fea = test_fea.drop('prob', axis=1)
test = test_fea.iloc[:, 2:]
y = train_fea.iloc[:, -1]

train.shape, test.shape, y.shape

((19636288, 40), (3169697, 40), (19636288,))

In [71]:
KF = StratifiedKFold(n_splits=kfold, random_state=seed, shuffle=True)
feat_imp_lgb = list()

oof_lgb = np.zeros(len(train))
predictions_lgb = np.zeros((len(test)))
print(train.shape[1])

# 模型训练
for fold_, (trn_idx, val_idx) in enumerate(KF.split(train.values, y.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx], label=y.iloc[trn_idx])
    val_data = lgb.Dataset(train.iloc[val_idx], label=y.iloc[val_idx])
    
    num_round = 3000
    clf = lgb.train(
        lgb_params,
        trn_data,
        num_round,
        valid_sets=[trn_data, val_data],
        verbose_eval=100,
        early_stopping_rounds=100,
    )

    oof_lgb[val_idx] = clf.predict(train.iloc[val_idx], num_iteration=clf.best_iteration)
    predictions_lgb[:] += clf.predict(test, num_iteration=clf.best_iteration) / kfold
    feat_imp_lgb.append(clf.feature_importance())

print("AUC score: {}".format(metrics.roc_auc_score(y, oof_lgb)))
print("F1 score: {}".format(metrics.f1_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))
print("Precision score: {}".format(metrics.precision_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))
print("Recall score: {}".format(metrics.recall_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))

40
fold n°0
[LightGBM] [Warning] num_threads is set with nthread=-1, will be overridden by n_jobs=30. Current value: num_threads=30
Training until validation scores don't improve for 100 rounds
[300]	training's auc: 0.722561	valid_1's auc: 0.719763
[600]	training's auc: 0.731618	valid_1's auc: 0.726326
[900]	training's auc: 0.737944	valid_1's auc: 0.730412


KeyboardInterrupt: 

In [ ]:
print("AUC score: {}".format(metrics.roc_auc_score(y, oof_lgb)))
print("F1 score: {}".format(metrics.f1_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))
print("Precision score: {}".format(metrics.precision_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))
print("Recall score: {}".format(metrics.recall_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))

In [ ]:
np.save('predictions_lgb.npy', predictions_lgb)
np.save('oof_lgb.npy', oof_lgb)
np.save('feat_imp_lgb.npy', feat_imp_lgb)

In [ ]:
predictions_lgb.shape, oof_lgb.shape 

In [ ]:
test_fea['prob'] = predictions_lgb
train_fea['prob'] = oof_lgb

In [ ]:
test_preds_lgb = []
test_fea_len = test_fea['index'].nunique()
for ind, test_fea_index in tqdm(test_fea[['vid', 'index', 'prob']].groupby('index'), total=test_fea_len):
    # 对召回的10个从大到小排序
    test_fea_index = test_fea_index.sort_values(by='prob', ascending=False)['vid'].values.tolist()
    if len(test_fea_index) != 10:
        print(ind) 
    # 对完整召回的100个剩下的其他的id，不在10个里的话就补充进去
    preds_ind_origin = test_preds[ind]
    for vid in preds_ind_origin:
        if vid not in test_fea_index:
            test_fea_index.append(vid)
    if len(test_fea_index) != 100:
        print('error', ind) 
    test_preds_lgb.append(test_fea_index)

In [ ]:
df_test['next_item_prediction'] = test_preds_lgb
df_test['next_item_prediction'].apply(len).describe()

In [ ]:
df_test[['locale', 'next_item_prediction']].to_parquet('output/submission_task1_lgb_{}_{}.parquet'.format(seed, train.shape[1]), engine='pyarrow')

xgb

In [ ]:
seed = 2023
kfold = 5

In [72]:
xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'random_state': seed,
    'tree_method': 'hist',
    'booster': 'gbtree',
    'n_estimators': 1000, 
    'learning_rate': 0.1,
    'max_depth': 6,
    'min_child_weight': 12, 
    'subsample': 0.8,
    'colsample_bytree': 1.0, 
    'alpha': 0.7,
    'lambda': 0.1, 
    'gamma': 0.1, 
    "nthread": -1
}

In [232]:
xgb_params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'gamma': 1,
    'min_child_weight': 1.5,
    'max_depth': 6,
    'lambda': 10,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'colsample_bylevel': 0.7,
    'eta': 0.05,
    'tree_method': 'exact',
    'seed': seed,
    'nthread': -1
}

In [233]:
if 'prob' in train_fea.columns:
    train_fea = train_fea.drop('prob', axis=1)
train = train_fea.iloc[:, 0: -1]
if 'prob' in test_fea.columns:
    test_fea = test_fea.drop('prob', axis=1)
test = test_fea.iloc[:, 2:]
y = train_fea.iloc[:, -1]

train.shape, test.shape, y.shape

((19636288, 40), (3169697, 40), (19636288,))

In [234]:
KF = StratifiedKFold(n_splits=kfold, random_state=seed, shuffle=True)
feat_imp_xgb = list()

oof_xgb = np.zeros(len(train))
predictions_xgb = np.zeros((len(test)))
print(train.shape[1])

# 模型训练
for fold_, (trn_idx, val_idx) in enumerate(KF.split(train.values, y.values)):
    print("fold n°{}".format(fold_))
    X_train, X_val = train.iloc[trn_idx], train.iloc[val_idx]
    y_train, y_val = y.iloc[trn_idx], y.iloc[val_idx]
    trn_data = xgb.DMatrix(X_train, label=y_train)
    val_data = xgb.DMatrix(X_val, label=y_val)

    num_round = 3000
    clf = xgb.train(
        xgb_params,
        trn_data,
        num_round,
        evals = [(trn_data, 'train'), (val_data, 'val')],
        verbose_eval=100,
        early_stopping_rounds=100,
    )

    oof_xgb[val_idx] = clf.predict(val_data, iteration_range=(0, clf.best_iteration))
    predictions_xgb[:] += clf.predict(xgb.DMatrix(test), iteration_range=(0, clf.best_iteration)) / kfold
    feat_imp_xgb.append(clf.get_score())

40
fold n°0
[0]	train-logloss:0.67074	val-logloss:0.67074
[100]	train-logloss:0.38975	val-logloss:0.38982
[200]	train-logloss:0.38554	val-logloss:0.38568
[300]	train-logloss:0.38430	val-logloss:0.38451
[400]	train-logloss:0.38344	val-logloss:0.38374
[500]	train-logloss:0.38281	val-logloss:0.38321
[600]	train-logloss:0.38232	val-logloss:0.38282


KeyboardInterrupt: 

In [ ]:
print("AUC score: {}".format(metrics.roc_auc_score(y, oof_xgb)))
print("F1 score: {}".format(metrics.f1_score(y, [1 if i >= 0.5 else 0 for i in oof_xgb])))
print("Precision score: {}".format(metrics.precision_score(y, [1 if i >= 0.5 else 0 for i in oof_xgb])))
print("Recall score: {}".format(metrics.recall_score(y, [1 if i >= 0.5 else 0 for i in oof_xgb])))

AUC score: 0.7459473405030757
F1 score: 0.023517479921286524
Precision score: 0.6333548699725408
Recall score: 0.01198118015739832


In [ ]:
predictions_xgb.shape, oof_xgb.shape 

(3169697,)

In [ ]:
np.save('predictions_xgb_newparam.npy', predictions_xgb)
np.save('oof_xgb_newparam.npy', oof_xgb)
np.save('feat_imp_xgb_newparam.npy', feat_imp_xgb)

In [ ]:
test_fea['prob'] = predictions_xgb
train_fea['prob'] = oof_xgb

In [ ]:
test_preds_xgb = []
test_fea_len = test_fea['index'].nunique()
for ind, test_fea_index in tqdm(test_fea[['vid', 'index', 'prob']].groupby('index'), total=test_fea_len):
    # 对召回的10个从大到小排序
    test_fea_index = test_fea_index.sort_values(by='prob', ascending=False)['vid'].values.tolist()
    if len(test_fea_index) != 10:
        print(ind) 
    # 对完整召回的100个剩下的其他的id，不在10个里的话就补充进去
    preds_ind_origin = test_preds[ind]
    for vid in preds_ind_origin:
        if vid not in test_fea_index:
            test_fea_index.append(vid)
    if len(test_fea_index) != 100:
        print('error', ind) 
    test_preds_xgb.append(test_fea_index)

 43%|████▎     | 135232/316971 [03:01<03:42, 817.26it/s] 

135090


 55%|█████▌    | 175898/316971 [03:54<02:35, 904.79it/s] 

175732


 56%|█████▌    | 177975/316971 [03:57<03:08, 736.00it/s]

...

 61%|██████    | 193151/316971 [04:18<03:06, 665.29it/s] 

193020


 72%|███████▏  | 227628/316971 [05:05<02:02, 728.91it/s] 

227523


100%|██████████| 316971/316971 [07:07<00:00, 741.44it/s] 


In [ ]:
test_fea[test_fea['index'] == 0][['vid', 'index', 'prob']]

,vid,index,prob
0,B099NS1XPG,0,0.292281
1,B08496TCCQ,0,0.000004
2,B01BVG1XJS,0,0.000004
3,B099NR3X6D,0,0.286143
4,B09WDSH4CD,0,0.307145
5,B0BHWQJJKW,0,0.274711
6,B07LG5T3V9,0,0.155305
7,B0BGL7KC2D,0,0.264385
8,B0B53KBXR8,0,0.213375
9,B07QPV9Z7X,0,0.000707


In [ ]:
df_test['next_item_prediction'] = test_preds_xgb
df_test['next_item_prediction'].apply(len).describe()

count    316971.0
mean        100.0
std           0.0
min         100.0
25%         100.0
50%         100.0
75%         100.0
max         100.0
Name: next_item_prediction, dtype: float64

In [ ]:
df_test[['locale', 'next_item_prediction']].to_parquet('output/submission_task1_xgb_{}_{}_newparam.parquet'.format(seed, train.shape[1]), engine='pyarrow')

cbc

In [205]:
seed = 2023
kfold = 5

In [206]:
cbc_params = {
    'random_state': seed,
    'loss_function': 'Logloss',
    'eval_metric': 'Logloss',
    'verbose': False,
    'learning_rate': 0.05,
    'depth': 5,
    'rsm': 0.2020238568794654,
    # 'min_data_in_leaf': 255,
    # 'l2_leaf_reg': 5,
    # 'subsample': 0.7,
    # 'use_best_model': True,
    # 'max_leaves': 12,
    # 'metric_period': 500,
    'n_estimators': 3000,
    'thread_count': -1
}

In [219]:
if 'prob' in train_fea.columns:
    train_fea = train_fea.drop('prob', axis=1)
train = train_fea.iloc[:, 0: -1]
if 'prob' in test_fea.columns:
    test_fea = test_fea.drop('prob', axis=1)
test = test_fea.iloc[:, 2:]
y = train_fea.iloc[:, -1]

train.shape, test.shape, y.shape

((19636288, 40), (3169697, 40), (19636288,))

In [225]:
KF = StratifiedKFold(n_splits=kfold, random_state=seed, shuffle=True)
feat_imp_cbc = list()

oof_cbc = np.zeros(len(train))
predictions_cbc = np.zeros((len(test)))
print(train.shape[1])

model = CatBoostClassifier(**cbc_params)

# 模型训练
for fold_, (trn_idx, val_idx) in enumerate(KF.split(train.values, y.values)):
    print("fold n°{}".format(fold_))

    X_train, X_val = train.iloc[trn_idx], train.iloc[val_idx]
    y_train, y_val = y.iloc[trn_idx], y.iloc[val_idx]
    
    model.fit(X_train, y_train, eval_set=(X_val, y_val), 
              cat_features=cat_features,
              early_stopping_rounds=100, verbose=100, use_best_model=True)

    oof_cbc[val_idx] += (model.predict_proba(X_val)[:, 1])
    predictions_cbc += (model.predict_proba(test)[:, 1]) / kfold
    feat_imp_cbc.append(model.feature_importances_)

40
fold n°0
0:	learn: 0.6357881	test: 0.6358288	best: 0.6358288 (0)	total: 9.03s	remaining: 7h 31m 34s
100:	learn: 0.3916500	test: 0.3912687	best: 0.3912687 (100)	total: 10m 39s	remaining: 5h 5m 42s
200:	learn: 0.3515971	test: 0.3049604	best: 0.3049604 (200)	total: 23m 27s	remaining: 5h 26m 36s
300:	learn: 0.3452265	test: 0.2908280	best: 0.2908280 (300)	total: 33m 7s	remaining: 4h 56m 57s
400:	learn: 0.3359115	test: 0.2645723	best: 0.2645723 (400)	total: 46m 44s	remaining: 5h 2m 58s
500:	learn: 0.3311241	test: 0.2492424	best: 0.2492424 (500)	total: 1h 11s	remaining: 5h 12s
600:	learn: 0.3302944	test: 0.2476811	best: 0.2476811 (600)	total: 1h 10m 31s	remaining: 4h 41m 31s
700:	learn: 0.3294730	test: 0.2453735	best: 0.2453735 (700)	total: 1h 21m 36s	remaining: 4h 27m 37s
800:	learn: 0.3291346	test: 0.2449336	best: 0.2449299 (799)	total: 1h 31m 16s	remaining: 4h 10m 35s
900:	learn: 0.3284237	test: 0.2425301	best: 0.2425301 (900)	total: 1h 41m 25s	remaining: 3h 56m 16s
1000:	learn: 0.32766

In [226]:
print("AUC score: {}".format(metrics.roc_auc_score(y, oof_cbc)))
print("F1 score: {}".format(metrics.f1_score(y, [1 if i >= 0.5 else 0 for i in oof_cbc])))
print("Precision score: {}".format(metrics.precision_score(y, [1 if i >= 0.5 else 0 for i in oof_cbc])))
print("Recall score: {}".format(metrics.recall_score(y, [1 if i >= 0.5 else 0 for i in oof_cbc])))

AUC score: 0.9452826396399912
F1 score: 0.5954382863379888
Precision score: 0.7165961409242333
Recall score: 0.5093246710072001


In [227]:
np.save('predictions_cbc.npy', predictions_cbc)
np.save('oof_cbc.npy', oof_cbc)
np.save('feat_imp_cbc.npy', feat_imp_cbc)

In [238]:
test_fea['prob'] = predictions_cbc
train_fea['prob'] = oof_cbc

In [229]:
test_preds_cbc = []
test_fea_len = test_fea['index'].nunique()
for ind, test_fea_index in tqdm(test_fea[['vid', 'index', 'prob']].groupby('index'), total=test_fea_len):
    # 对召回的10个从大到小排序
    test_fea_index = test_fea_index.sort_values(by='prob', ascending=False)['vid'].values.tolist()
    if len(test_fea_index) != 10:
        print(ind) 
    # 对完整召回的100个剩下的其他的id，不在10个里的话就补充进去
    preds_ind_origin = test_preds[ind]
    for vid in preds_ind_origin:
        if vid not in test_fea_index:
            test_fea_index.append(vid)
    if len(test_fea_index) != 100:
        print('error', ind) 
    test_preds_cbc.append(test_fea_index)

 43%|████▎     | 135194/316971 [03:15<04:21, 693.95it/s]

135090


 55%|█████▌    | 175811/316971 [04:11<03:41, 635.93it/s] 

175732


 56%|█████▌    | 178034/316971 [04:15<02:42, 857.02it/s]

...

 61%|██████    | 193153/316971 [04:36<03:11, 647.01it/s]

193020


 72%|███████▏  | 227693/316971 [05:27<01:52, 794.20it/s]

227523


100%|██████████| 316971/316971 [07:33<00:00, 698.99it/s] 


In [230]:
df_test['next_item_prediction'] = test_preds_cbc
df_test['next_item_prediction'].apply(len).describe()

count    316971.0
mean        100.0
std           0.0
min         100.0
25%         100.0
50%         100.0
75%         100.0
max         100.0
Name: next_item_prediction, dtype: float64

In [231]:
df_test[['locale', 'next_item_prediction']].to_parquet('output/submission_task1_cbc_{}_{}.parquet'.format(seed, train.shape[1]), engine='pyarrow')

In [241]:
train_fea[['label', 'prob']].head(20)

,label,prob
0,0.0,5.688426e-07
1,0.0,4.421515e-01
2,0.0,6.065247e-02
3,0.0,8.808528e-02
4,0.0,6.406443e-02
5,1.0,3.946522e-01
6,0.0,5.130541e-03
7,0.0,1.940774e-04
8,0.0,6.467994e-05
9,0.0,1.049645e-02


# 11111